In [1]:
import os
from pathlib import Path

In [2]:
import sys
print(sys.executable)


e:\my career life\Projects\AI projects\NLP-Text-Summarization\.venv\Scripts\python.exe


In [3]:
%pwd

'e:\\my career life\\Projects\\AI projects\\NLP-Text-Summarization\\research'

In [4]:
PROJECT_ROOT = Path.cwd().parent
os.chdir(PROJECT_ROOT)

In [5]:
%pwd

'e:\\my career life\\Projects\\AI projects\\NLP-Text-Summarization'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [9]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

e:\my career life\Projects\AI projects\NLP-Text-Summarization\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)
    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'],max_length=1024,truncation=True)

        target_encodings = self.tokenizer(text_target = example_batch['summary'], max_length = 128, truncation = True )
        
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_tf = dataset_samsum.map(self.convert_examples_to_features,batched =True)
        dataset_samsum_tf.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [13]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2026-02-06 20:05:19,693: INFO: common: yaml file: config\config.yaml loaded successfully]
[2026-02-06 20:05:19,697: INFO: common: yaml file: params.yaml loaded successfully]
[2026-02-06 20:05:19,699: INFO: common: created directory at: artifacts]
[2026-02-06 20:05:19,702: INFO: common: created directory at: artifacts/data_transformation]
[2026-02-06 20:05:20,400: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/config.json "HTTP/1.1 307 Temporary Redirect"]


[2026-02-06 20:05:20,405: WARNING: _http: Warning: You are sending unauthenticated requests to the HF Hub. Please set a HF_TOKEN to enable higher rate limits and faster downloads.]
[2026-02-06 20:05:20,557: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/config.json "HTTP/1.1 200 OK"]
[2026-02-06 20:05:20,939: INFO: _client: HTTP Request: HEAD https://huggingface.co/google/pegasus-cnn_dailymail/resolve/main/tokenizer_config.json "HTTP/1.1 307 Temporary Redirect"]
[2026-02-06 20:05:21,079: INFO: _client: HTTP Request: HEAD https://huggingface.co/api/resolve-cache/models/google/pegasus-cnn_dailymail/40d588fdab0cc077b80d950b300bf66ad3c75b92/tokenizer_config.json "HTTP/1.1 200 OK"]
[2026-02-06 20:05:21,442: INFO: _client: HTTP Request: GET https://huggingface.co/api/models/google/pegasus-cnn_dailymail/tree/main/additional_chat_templates?recursive=false&expand=false "HTTP/1.1 404 Not Foun

Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 58937.71 examples/s]
